# Final Project

1. Get data 
    * coal plants states, cities
    * death rate in plant city
    * state death rate
1. compare to state avg
1. compare state avg to country avg

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

### import data

In [2]:
data = pd.read_excel('power_plants_and_communities.xls') # data from 2019
coal_df = data[data['Flag indicating if the plant burned or generated any amount of coal'] == 'Yes'] # only if plant burns coal
no_coal_df = data[data['Flag indicating if the plant burned or generated any amount of coal'] != 'Yes'] # only if plant does not burns coal
mortality_df = pd.read_excel('2022 County Health Rankings Data.xlsx', sheet_name=5)


In [3]:
mortality_df = mortality_df.rename(columns=mortality_df.iloc[0]).drop(mortality_df.index[0]).reset_index(drop = True)

#### Sort Data

In [4]:
states = pd.array(coal_df['Plant state abbreviation'])
all_coal_states = set(np.array(coal_df['Plant state abbreviation']))
coal_plants = np.array(coal_df['Plant name'])
all_coal_county = np.array(coal_df['Plant county name'])
coal_plant_amnt = len(coal_plants)
filtered_coal_states = []
filtered_coal_county = []

no_coal_states = no_coal_df['Plant state abbreviation']
no_coal_plants = no_coal_df['Plant name']
no_coal_county = no_coal_df['Plant county name']
no_coal_plant_amnt = len(no_coal_plants)


* Filter results between the death rates and plant data

In [5]:
for y in range(len(mortality_df['County'])):
    if mortality_df['County'][y] in all_coal_county:
        filtered_coal_county.append(mortality_df['County'][y])
    elif mortality_df['State'][y] not in all_coal_states:
        filtered_coal_states.append(mortality_df['State'][y])


In [ ]:
filtered_coal_states